<a href="https://colab.research.google.com/github/kgautam2103/Meetup_automation/blob/main/Update_meetup_events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
client_key=""

client_redirect_url="https://us.sahajayoga.org/"

authorize_url = "https://secure.meetup.com/oauth2/authorize?client_id="+client_key+"&response_type=code&redirect_uri="+client_redirect_url

print(authorize_url)

https://secure.meetup.com/oauth2/authorize?client_id=emgo5alo5672bdgr1k4bp6sn57&response_type=code&redirect_uri=https://us.sahajayoga.org/


In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
from dateutil import tz

def get_access_refresh_token_from_code(access_code,client_secret, grant_type_code):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_code+"&redirect_uri="+client_redirect_url+"&code="+access_code


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


def get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_refresh+"&refresh_token="+refresh_token


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


access_code = input("please enter the access code recieved ")

client_secret=""

grant_type_code = "authorization_code"

grant_type_refresh = "refresh_token"

meetup_api_url = "https://api.meetup.com/gql"

#sheet_name = input('enter the start time in UTC in format 2022-12-31T13:45 : ')

access_token,refresh_token = get_access_refresh_token_from_code(access_code,client_secret, grant_type_code)

please enter the access code recieved a9fde1b33bc905e2328b00dc8fa34074
{'access_token': '4be61a3cfba25e2d0fe374a168d88b5c', 'refresh_token': 'a05ff2d262d51d18fce38870f5bd8c18', 'token_type': 'bearer', 'expires_in': 3600}


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/18OKklMTyZLG8pHFlSUaIAKkyJsPcB5vER-atJVmgxa4/edit#gid=0')

current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_event_output_meetup'

counter=1

#Naol_start_time = "2023-11-11T21:45"
Naol_start_time = "2023-11-05T09:45"

from_tz = "America/New_York"

# create and Open our new sheet and add some data.
sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['group_url','group_name','event_id','event_url','timezone','error'],1)

sheet_name = input('enter the name of input sheet : ')
input_sheet = wb.worksheet(sheet_name)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['group_name',	'event_id'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])


for index,row in inputdata_df.iterrows():
  counter = counter+1
  created_event_id=""
  created_event_url=""
  created_event_error=""
  access_token,refresh_token = get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh)
  authorization = 'Bearer ' + access_token
  auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

  print("access_token "+access_token)
  print("refresh_token "+refresh_token)
  cell_value_list = []
  group_url = str(row['group_name'])
  event_id = row['event_id']

  group_name = group_url.split("/")[3]


  group_query = """query($grp_url : String!){
              groupByUrlname(urlname: $grp_url){
              id
              timezone
              }
            }"""

  variables_group_query = {"grp_url":group_name}

  gql_group_query_response = requests.post(meetup_api_url,json={'query': group_query, 'variables': variables_group_query}, headers=auth_headers)

  print(gql_group_query_response.text)

  group_id = gql_group_query_response.json()['data']['groupByUrlname']['id']
  print(group_id)
  grp_timezone = gql_group_query_response.json()['data']['groupByUrlname']['timezone']
  print(grp_timezone)


  from_zone = tz.gettz(from_tz)
  to_zone = tz.gettz(grp_timezone)

  start_timestamp = datetime.strptime(Naol_start_time,"%Y-%m-%dT%H:%M")
  print(start_timestamp)
  old_timezone = start_timestamp.replace(tzinfo=from_zone)
  new_timezone = old_timezone.astimezone(to_zone)
  print(old_timezone)
  print(new_timezone)
  date_input = datetime.strftime(new_timezone,"%Y-%m-%dT%H:%M")
  print(date_input)

  edit_event_query = """
      mutation($edit_event_input: EditEventInput!) {
    editEvent(input: $edit_event_input) {
      event {
        id
      }
      errors {
        message
        code
        field
      }
    }
  }
  """


  edit_event_variables = {
    "edit_event_input": {
      "eventId": event_id,
      "startDateTime":date_input,
      "applyToSeries":True



    }
  }

  edit_event_mutation_response = requests.post(meetup_api_url, json={'query': edit_event_query, 'variables': edit_event_variables}, headers=auth_headers)

  print(edit_event_mutation_response.text)

  try:
    edited_event_error=edit_event_mutation_response.json()['data']['editEvent']['errors'][0]['message']
    edited_event_id=""
    edited_event_url=""
  except:
    edited_event_error=""
    edited_event_id=edit_event_mutation_response.json()['data']['editEvent']['event']['id']
    edited_event_url=group_url+"events/"+created_event_id+"/"

  cell_value_list = [group_url,group_name,edited_event_id,edited_event_url,grp_timezone,created_event_error]
  print(cell_value_list)
  output_sheet.insert_row(cell_value_list,counter)


enter the name of input sheet : NAOL
{'access_token': '3aa1d4e6a9dc367084dbee23b0d1be90', 'refresh_token': 'a05ff2d262d51d18fce38870f5bd8c18', 'token_type': 'bearer', 'expires_in': 3600}
access_token 3aa1d4e6a9dc367084dbee23b0d1be90
refresh_token a05ff2d262d51d18fce38870f5bd8c18
{"data":{"groupByUrlname":{"id":"2946382","timezone":"America/Phoenix"}}}

2946382
America/Phoenix
2023-11-05 09:45:00
2023-11-05 09:45:00-05:00
2023-11-05 07:45:00-07:00
2023-11-05T07:45
{"data":{"editEvent":{"event":{"id":"296771421"},"errors":[]}}}

['https://www.meetup.com/Phoenix-Sahaja-Yoga-Meditation/', 'Phoenix-Sahaja-Yoga-Meditation', '296771421', 'https://www.meetup.com/Phoenix-Sahaja-Yoga-Meditation/events//', 'America/Phoenix', '']
{'access_token': '68c4f13a8b0126bb8782c62b11a61169', 'refresh_token': 'a05ff2d262d51d18fce38870f5bd8c18', 'token_type': 'bearer', 'expires_in': 3600}
access_token 68c4f13a8b0126bb8782c62b11a61169
refresh_token a05ff2d262d51d18fce38870f5bd8c18
{"data":{"groupByUrlname":{"i